In [23]:
# Module was created referencing '18-Unsupervised-Learning-Day2-Solved'

# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from datetime import datetime

# Clean up CMO-Monthly-prices

In [24]:
#Pull
indices_df = pd.read_csv(r'../Resources/data/master/CMO-Monthly-Indices.csv', index_col = 0)
indices_df.index.name = 'Month'
indices_df.head()

#Select only the data for 2016-2021
years = ("2016|2017|2018|2019|2020|2021")
indices_df = indices_df[indices_df.index.str.contains(years)]
indices_df.to_csv('clean_indices.csv')

# Clean up Contracts csv

In [25]:
# Pull Energy contracts master file

awards_df = pd.read_csv(r'../Resources/data/master/Contracts_PrimeAwardSummaries_2016-2022_NAICS_21.csv', index_col = 0)
awards_df

# Count Null Values
#for column in awards_df.columns:
    #print(f"Column {column} has {awards_df[column].isnull().sum()} null values")

#Select only data for 2016-2021
years = ("2016|2017|2018|2019|2020|2021")
awards_df = awards_df[awards_df.award_base_action_date.str.contains(years)]
awards_df.head()

,award_base_action_date,period_of_performance_start_date,period_of_performance_current_end_date,period_of_performance_potential_end_date,funding_sub_agency_name,recipient_name,recipient_city_name,recipient_county_name,recipient_state_code,product_or_service_code,product_or_service_code_description,naics_code,naics_description
total_obligated_amount,,,,,,,,,,,,,
10000.00,9/26/2016,9/26/2016,10/31/2016 0:00,10/31/2016 0:00,DEPT OF THE NAVY,"AMERIGAS PROPANE, L.P.",KING OF PRUSSIA,MONTGOMERY,PA,6830,GASES: COMPRESSED AND LIQUEFIED,211112,NATURAL GAS LIQUID EXTRACTION
1384.98,2/18/2016,2/18/2016,4/5/2016 0:00,4/5/2016 0:00,FEDERAL PRISON SYSTEM,"BRAATEN SAND & GRAVEL, INC",BYRON,OLMSTED,MN,5610,"MINERAL CONSTRUCTION MATERIALS, BULK",212321,CONSTRUCTION SAND AND GRAVEL MINING
542491.46,5/12/2017,5/12/2017,7/11/2017 0:00,8/10/2017 0:00,DEFENSE LOGISTICS AGENCY,"POND CONSTRUCTORS, INC",NORCROSS,GWINNETT,GA,Z1NA,MAINTENANCE OF FUEL SUPPLY FACILITIES,213112,SUPPORT ACTIVITIES FOR OIL AND GAS OPERATIONS
21764.63,3/4/2016,3/4/2016,3/17/2016 0:00,3/17/2016 0:00,"STATE, DEPARTMENT OF",MISCELLANEOUS FOREIGN AWARDEES,WASHINGTON,DISTRICT OF COLUMBIA,DC,H999,OTHER QC/TEST/INSPECT- MISCELLANEOUS,212321,CONSTRUCTION SAND AND GRAVEL MINING
256800.00,12/5/2020,12/5/2020,12/27/2020 0:00,12/27/2020 0:00,DEPT OF THE ARMY,FOREIGN AWARDEES (UNDISCLOSED),WASHINGTON,DISTRICT OF COLUMBIA,DC,5610,"MINERAL CONSTRUCTION MATERIALS, BULK",212319,OTHER CRUSHED AND BROKEN STONE MINING AND QUAR...


In [26]:
awards_df.funding_sub_agency_name.unique()

array(['DEPT OF THE NAVY', 'FEDERAL PRISON SYSTEM',
       'DEFENSE LOGISTICS AGENCY', 'STATE, DEPARTMENT OF',
       'DEPT OF THE ARMY',
       'BUREAU OF INDIAN AFFAIRS AND BUREAU OF INDIAN EDUCATION',
       'DEPT OF THE AIR FORCE', 'PUBLIC BUILDINGS SERVICE',
       'INDIAN HEALTH SERVICE', 'MARITIME ADMINISTRATION',
       'NATIONAL PARK SERVICE', 'BUREAU OF RECLAMATION',
       'FEDERAL ACQUISITION SERVICE',
       'FEDERAL PRISON SYSTEM / BUREAU OF PRISONS',
       'J. F. KENNEDY CENTER FOR THE PERFORMING ARTS',
       'BUREAU OF LAND MANAGEMENT', 'FOREST SERVICE',
       'U.S. FISH AND WILDLIFE SERVICE', 'ENERGY, DEPARTMENT OF',
       'VETERANS AFFAIRS, DEPARTMENT OF',
       'U.S. ARMY CORPS OF ENGINEERS - CIVIL PROGRAM FINANCING ONLY',
       'U.S. COAST GUARD', 'AGRICULTURAL RESEARCH SERVICE',
       'FEDERAL EMERGENCY MANAGEMENT AGENCY',
       'FEDERAL HIGHWAY ADMINISTRATION',
       'AGENCY FOR INTERNATIONAL DEVELOPMENT',
       'NATIONAL TELECOMMUNICATIONS AND INFORMATI

In [27]:
#Convert the date columns to datetime format to match indices_df
columns = ['Month']
awards_df.award_base_action_date = pd.to_datetime(awards_df.award_base_action_date)

awards_df.award_base_action_date = awards_df.award_base_action_date.dt.strftime('%YM%m')


# Create new df with only necessary columns
clean_contracts_df = awards_df[['award_base_action_date','product_or_service_code_description', 'naics_description', 'recipient_name','funding_sub_agency_name']]

clean_contracts_df.rename(columns = {'award_base_action_date':'Month'}, inplace = True)

C:\Users\Chapm\.conda\envs\PythonData\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
# Save as CSV
clean_contracts_df.to_csv('../Resources/Data/clean/clean_contracts.csv')

clean_contracts_df.funding_sub_agency_name.unique()

array(['DEPT OF THE NAVY', 'FEDERAL PRISON SYSTEM',
       'DEFENSE LOGISTICS AGENCY', 'STATE, DEPARTMENT OF',
       'DEPT OF THE ARMY',
       'BUREAU OF INDIAN AFFAIRS AND BUREAU OF INDIAN EDUCATION',
       'DEPT OF THE AIR FORCE', 'PUBLIC BUILDINGS SERVICE',
       'INDIAN HEALTH SERVICE', 'MARITIME ADMINISTRATION',
       'NATIONAL PARK SERVICE', 'BUREAU OF RECLAMATION',
       'FEDERAL ACQUISITION SERVICE',
       'FEDERAL PRISON SYSTEM / BUREAU OF PRISONS',
       'J. F. KENNEDY CENTER FOR THE PERFORMING ARTS',
       'BUREAU OF LAND MANAGEMENT', 'FOREST SERVICE',
       'U.S. FISH AND WILDLIFE SERVICE', 'ENERGY, DEPARTMENT OF',
       'VETERANS AFFAIRS, DEPARTMENT OF',
       'U.S. ARMY CORPS OF ENGINEERS - CIVIL PROGRAM FINANCING ONLY',
       'U.S. COAST GUARD', 'AGRICULTURAL RESEARCH SERVICE',
       'FEDERAL EMERGENCY MANAGEMENT AGENCY',
       'FEDERAL HIGHWAY ADMINISTRATION',
       'AGENCY FOR INTERNATIONAL DEVELOPMENT',
       'NATIONAL TELECOMMUNICATIONS AND INFORMATI

# Clean up CMO prices

In [37]:
#Pull
prices_df = pd.read_csv(r'../Resources/data/master/CMO-Monthly-Prices.csv', index_col = 0)
prices_df.index.name = 'Month'
prices_df.head()

#Select only the data for 2016-2021
years = ("2016|2017|2018|2019|2020|2021")
prices_df = prices_df[prices_df.index.str.contains(years, na=False)]
prices_df.to_csv('clean_prices.csv')

In [38]:
prices_df

,"Crude oil, average","Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI","Coal, Australian","Coal, South African **","Natural gas, US","Natural gas, Europe **","Liquefied natural gas, Japan",Natural gas index,...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
Month,,,,,,,,,,,,,,,,,,,,,
2016M01,29.78,30.80,27.00,31.54,49.02,49.09,2.27,4.40,8.40,54.22,...,1481.10,41.88,4471.79,1646.20,13808.08,8507.29,1520.36,1097.91,855.25,14.11
2016M02,31.03,33.20,29.50,30.39,50.27,50.33,1.96,3.97,8.56,48.43,...,1531.26,46.83,4598.62,1765.75,15610.14,8298.50,1709.85,1199.50,919.35,15.17
2016M03,37.34,39.07,35.18,37.77,52.21,52.57,1.70,3.91,7.74,44.49,...,1531.01,56.20,4953.80,1802.19,16897.60,8717.25,1801.69,1245.14,967.73,15.47
2016M04,40.75,42.25,39.04,40.96,50.69,52.35,1.90,3.97,6.82,46.63,...,1571.23,60.92,4872.74,1732.27,17032.71,8878.86,1855.37,1242.26,994.19,16.36
2016M05,45.94,47.13,43.95,46.73,51.31,54.24,1.92,4.33,6.27,48.17,...,1550.63,55.13,4694.54,1707.80,16706.95,8660.35,1869.03,1260.95,1035.86,16.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021M08,68.87,70.02,68.85,67.73,169.58,137.92,4.05,15.43,10.80,129.82,...,2602.99,162.16,9370.14,2414.47,35024.00,19141.30,2987.95,1785.28,1008.73,23.99
2021M09,72.80,74.60,72.24,71.56,185.69,146.05,5.11,22.84,11.44,178.40,...,2834.56,124.52,9324.71,2248.30,34887.30,19376.88,3036.02,1775.14,973.19,23.19
2021M10,82.06,83.65,81.22,81.32,224.51,199.65,5.48,31.05,12.38,222.53,...,2934.39,122.91,9829.22,2344.83,37722.92,19362.39,3359.91,1776.85,1017.10,23.41
